In [1]:
# Estudiante: Ardanny Romero

# Parte 1: Importación de Librerías Principales

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time # Para hacer pausas entre peticiones
import sqlite3
import redis

In [1]:
# Parte 2: Cargar los datasets en pandas
# Parte 1: Volver a cargar los datasets en pandas
# Fuente 1: Kaggle - Video Game Sales: https://www.kaggle.com/datasets/gregorut/videogamesales
# Fuente 2: Kaggle - IMDb Movies extensive dataset: https://www.kaggle.com/datasets/rajugc/imdb-movies-dataset-based-on-genre
# Fuente 3: Yelp Open Dataset: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset/data

# Carga de Datos de Videojuegos y Cine
try:
    # Dataset de ventas de videojuegos
    df_vgsales = pd.read_csv('vgsales.csv')
    print("Dataset de ventas de videojuegos cargado exitosamente.")
    print(df_vgsales.head())

    # Dataset de de películas
    df_movies = pd.read_csv('IMDb_movies.csv', encoding='latin1')
    print("\nDataset de películas de IMDb cargado exitosamente.")
    print(df_movies.head())

except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo. Asegúrate de que el nombre del archivo es correcto y que lo has subido a Colab.")
    print(e)


# Carga de Datos de Restaurantes (Yelp)
try:
    # Usamos lines=True para leer el archivo correctamente.
    df_yelp_reviews = pd.read_json('yelp_academic_dataset_review.json', lines=True, nrows=500000)
    print("\nDataset de reseñas de Yelp cargado exitosamente.")
    # Nos interesan principalmente el texto de la reseña y las estrellas
    df_yelp_reviews_sample = df_yelp_reviews[['text', 'stars']]
    print(df_yelp_reviews_sample.head())

except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo de Yelp. Asegúrate de que el nombre del archivo es correcto.")
    print(e)

Dataset de ventas de videojuegos cargado exitosamente.
   Rank                      Name Platform    Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006.0        Sports  Nintendo   
1     2         Super Mario Bros.      NES  1985.0      Platform  Nintendo   
2     3            Mario Kart Wii      Wii  2008.0        Racing  Nintendo   
3     4         Wii Sports Resort      Wii  2009.0        Sports  Nintendo   
4     5  Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0     41.49     29.02      3.77         8.46         82.74  
1     29.08      3.58      6.81         0.77         40.24  
2     15.85     12.88      3.79         3.31         35.82  
3     15.75     11.01      3.28         2.96         33.00  
4     11.27      8.89     10.22         1.00         31.37  

Dataset de películas de IMDb cargado exitosamente.
    movie_id                         movie_name  year cert

In [8]:
# Parte 3: Web Scraping de Metacritic

# --- Fuente de Datos (Aporte de Persona 1) ---
# Fuente 4: Metacritic - PC Game Reviews: https://www.metacritic.com/browse/game/

# URL base sin el número de página
base_url = "https://www.metacritic.com/browse/game/pc/all/all-time/metascore/"

# Para simular que es un navegador para evitar bloqueos
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Lista para guardar todos los datos de los juegos extraídos
all_games_data = []

# Número de páginas que quieres extraer
PAGINAS_A_EXTRAER = 100

print(f"Iniciando scraping de las primeras {PAGINAS_A_EXTRAER} páginas de Metacritic...")

# Bucle para recorrer las páginas (Metacritic empieza en la página 0)
for page_num in range(PAGINAS_A_EXTRAER):
    # Construcción de la URL completa para la página actual
    url = f"{base_url}?page={page_num}"
    
    print(f"Extrayendo datos de: {url}")
    
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Busca todos los contenedores de los juegos
        games = soup.find_all('div', class_='c-finderProductCard-game')

        for game in games:
            # Extracción de Título
            # El título está en una etiqueta <a> dentro del div principal de la tarjeta
            title_element = game.find('h3', class_='c-finderProductCard_titleHeading')
            title = title_element.text.strip().split('.', 1)[-1].strip() if title_element else "No Title"

            # Extracción de Metascore
            score_element = game.find('div', class_='c-siteReviewScore')
            score = score_element.text.strip() if score_element else "No Score"
            
            # Extracción de más datos
            # Fecha de lanzamiento
            date_element = game.find('div', class_='c-finderProductCard_meta').find('span')
            release_date = date_element.text.strip() if date_element else "No Date"

            # Resumen del juego
            summary_element = game.find('div', class_='c-finderProductCard_description')
            summary = summary_element.text.strip() if summary_element else "No Summary"
            
            # URL del juego
            link_element = game.find('a', class_='c-finderProductCard_container')
            link = "https://www.metacritic.com" + link_element['href'] if link_element else "No Link"

            # Se añade los datos a la lista
            all_games_data.append({
                'title': title, 
                'metascore': score,
                'release_date': release_date,
                'summary': summary,
                'url': link
            })
    
    else:
        print(f"Error al conectar con la página {page_num}. Código de estado: {response.status_code}")

    # Pausa para no sobrecargar el servidor
    time.sleep(1)

# Se convierte la lista de datos a un DataFrame de Pandas
df_metacritic = pd.DataFrame(all_games_data)

print("\nScraping finalizado.")
print(f"Total de juegos extraídos: {len(df_metacritic)}")

# Mostramos los primeros resultados
print("\nPrimeros 5 juegos extraídos:")
print(df_metacritic.head())

Iniciando scraping de las primeras 100 páginas de Metacritic...
Extrayendo datos de: https://www.metacritic.com/browse/game/pc/all/all-time/metascore/?page=0
Extrayendo datos de: https://www.metacritic.com/browse/game/pc/all/all-time/metascore/?page=1
Extrayendo datos de: https://www.metacritic.com/browse/game/pc/all/all-time/metascore/?page=2
Extrayendo datos de: https://www.metacritic.com/browse/game/pc/all/all-time/metascore/?page=3
Extrayendo datos de: https://www.metacritic.com/browse/game/pc/all/all-time/metascore/?page=4
Extrayendo datos de: https://www.metacritic.com/browse/game/pc/all/all-time/metascore/?page=5
Extrayendo datos de: https://www.metacritic.com/browse/game/pc/all/all-time/metascore/?page=6
Extrayendo datos de: https://www.metacritic.com/browse/game/pc/all/all-time/metascore/?page=7
Extrayendo datos de: https://www.metacritic.com/browse/game/pc/all/all-time/metascore/?page=8
Extrayendo datos de: https://www.metacritic.com/browse/game/pc/all/all-time/metascore/?pag

In [4]:
df_metacritic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      24 non-null     object
 1   metascore  24 non-null     object
dtypes: object(2)
memory usage: 516.0+ bytes


In [10]:
# Padte 4: Conexión y Guardado en Bases de Datos (SQLite y Redis)

# Se crea una conexión a la base de datos
conn = sqlite3.connect('C:/Users/mtemox/Documents/mtemox/3er Semestre/Analisis de datos/Jupyter/Proyecto/proyecto_local.db')
print("\nConexión a SQLite establecida.")

# Se usa el DataFrame de ventas de videojuegos y lo guardamos en una tabla SQL
df_vgsales.to_sql('ventas_videojuegos', conn, if_exists='replace', index=False)
print("DataFrame 'df_vgsales' guardado en la tabla 'ventas_videojuegos' de SQLite.")

# Para verificar, se lee los datos de vuelta desde la base de datos
df_from_db = pd.read_sql('SELECT * FROM ventas_videojuegos LIMIT 5', conn)
print("\nVerificación - Primeros 5 registros leídos desde SQLite:")
print(df_from_db)

# Se cierra la conexión
conn.close()

# Para gaurdar en Redis

try:
    # Credenciales de tu base de datos Redis en la nube
    REDIS_HOST = 'redis-15895.c99.us-east-1-4.ec2.redns.redis-cloud.com'
    REDIS_PORT = 15895
    REDIS_PASSWORD = 'w3LHRp35ioUq92f4MzMG8c8PAHdv97fT'

    # Conexión a Redis
    r = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD, decode_responses=True)
    
    # Se prueba la conexión
    r.ping()
    print("\nConexión a Redis en la nube exitosa.")

    # Se gaurdan los datos de Metacritic en Redis.
    # Se usa un hash de Redis donde la clave es el título del juego y el valor es su score.
    for index, row in df_metacritic.iterrows():
        r.set(f"metacritic_game:{row['title']}", row['metascore'])

    print(f"{len(df_metacritic)} registros de Metacritic guardados en Redis.")

    # Para verificar se recupera un dato
    game_title_to_check = df_metacritic['title'][0]
    retrieved_score = r.get(f"metacritic_game:{game_title_to_check}")
    print(f"Verificación - Score de '{game_title_to_check}' recuperado de Redis: {retrieved_score}")

except redis.exceptions.ConnectionError as e:
    print("\nError de conexión a Redis: Verifica tus credenciales (host, puerto, contraseña).")
    print("Este error es esperado si no has reemplazado las credenciales de ejemplo.")
except Exception as e:
    print(f"\nOcurrió un error con Redis: {e}")


Conexión a SQLite establecida.
DataFrame 'df_vgsales' guardado en la tabla 'ventas_videojuegos' de SQLite.

Verificación - Primeros 5 registros leídos desde SQLite:
   Rank                      Name Platform    Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006.0        Sports  Nintendo   
1     2         Super Mario Bros.      NES  1985.0      Platform  Nintendo   
2     3            Mario Kart Wii      Wii  2008.0        Racing  Nintendo   
3     4         Wii Sports Resort      Wii  2009.0        Sports  Nintendo   
4     5  Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0     41.49     29.02      3.77         8.46         82.74  
1     29.08      3.58      6.81         0.77         40.24  
2     15.85     12.88      3.79         3.31         35.82  
3     15.75     11.01      3.28         2.96         33.00  
4     11.27      8.89     10.22         1.00         31.37  

In [36]:
# Celda 4: Conectar y cargar datos a SQL Server
import sqlalchemy

# Credenciales de conexión
server = 'proyecto-analisis-de-datos.database.windows.net'  # Ej: mi-servidor-epn.database.windows.net
database = 'ProyectoFinalDB'          # Ej: ProyectoAnalitica
username = 'admin-epn'                 # Ej: admin_user
password = 'Brandon.2.0'
driver = 'ODBC Driver 17 for SQL Server' # No cambiar esto

# --- Construir el motor de conexión con SQLAlchemy ---
# Este formato de URL es el que entiende SQLAlchemy para conectarse
connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"
engine = sqlalchemy.create_engine(connection_string)

print("Motor de SQLAlchemy creado. Intentando conectar...")

try:
    # --- Cargar cada DataFrame a una tabla en SQL Server ---
    # Usamos un bloque try-except para manejar posibles errores de conexión
    with engine.connect() as conn:
        print("¡Conexión a SQL Server exitosa!")

        # 1. Cargar df_vgsales_clean
        print("Cargando 'df_vgsales_clean' a la tabla 'VentasVideojuegos'...")
        df_vgsales_clean.to_sql('VentasVideojuegos', con=conn, if_exists='replace', index=False, chunksize=1000)
        print("Tabla 'VentasVideojuegos' cargada.")

        # 2. Cargar df_movies_clean
        print("Cargando 'df_movies_clean' a la tabla 'Peliculas'...")
        # Usamos chunksize para manejar la memoria con DataFrames grandes
        df_movies_clean.to_sql('Peliculas', con=conn, if_exists='replace', index=False, chunksize=1000)
        print("Tabla 'Peliculas' cargada.")

        # 3. Cargar df_yelp_clean
        print("Cargando 'df_yelp_clean' a la tabla 'ResenasYelp'...")
        df_yelp_clean.to_sql('ResenasYelp', con=conn, if_exists='replace', index=False, chunksize=1000)
        print("Tabla 'ResenasYelp' cargada.")

        # 4. Cargar df_metacritic_clean
        print("Cargando 'df_metacritic_clean' a la tabla 'ResenasMetacritic'...")
        df_metacritic_clean.to_sql('ResenasMetacritic', con=conn, if_exists='replace', index=False, chunksize=1000)
        print("Tabla 'ResenasMetacritic' cargada.")

        print("\n¡Proceso de carga a SQL Server completado!")

        # --- Verificación ---
        print("\nVerificando datos: Leyendo las 5 primeras filas de la tabla 'VentasVideojuegos' desde SQL Server...")
        df_from_sql = pd.read_sql("SELECT TOP 5 * FROM VentasVideojuegos", conn)
        print(df_from_sql)

except Exception as e:
    print(f"\nError durante la conexión o carga a SQL Server: {e}")
    print("Por favor, verifica tus credenciales, la configuración del firewall y que el servidor esté activo.")

Motor de SQLAlchemy creado. Intentando conectar...
¡Conexión a SQL Server exitosa!
Cargando 'df_vgsales_clean' a la tabla 'VentasVideojuegos'...
Tabla 'VentasVideojuegos' cargada.
Cargando 'df_movies_clean' a la tabla 'Peliculas'...
Tabla 'Peliculas' cargada.
Cargando 'df_yelp_clean' a la tabla 'ResenasYelp'...
Tabla 'ResenasYelp' cargada.
Cargando 'df_metacritic_clean' a la tabla 'ResenasMetacritic'...
Tabla 'ResenasMetacritic' cargada.

¡Proceso de carga a SQL Server completado!

Verificando datos: Leyendo las 5 primeras filas de la tabla 'VentasVideojuegos' desde SQL Server...
   Rank                      Name Platform  Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006        Sports  Nintendo   
1     2         Super Mario Bros.      NES  1985      Platform  Nintendo   
2     3            Mario Kart Wii      Wii  2008        Racing  Nintendo   
3     4         Wii Sports Resort      Wii  2009        Sports  Nintendo   
4     5  Pokemon Red/Pokemon Blue